### ***Installing the required dependencies***

In [ ]:
!pip install transformers datasets peft accelerate evaluate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

### ***Importing required modules for model training and evaluation***

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset
from peft import get_peft_model
from peft import LoraConfig
from peft import TaskType
from peft import prepare_model_for_kbit_training
import evaluate
import numpy as np
import torch

### ***Loading the dataset for English to Spanish translation task***

In [ ]:
dataset = load_dataset("opus_books", "en-es")
dataset = dataset["train"].train_test_split(test_size=0.2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/93470 [00:00<?, ? examples/s]

### ***Loading pre-trained model and tokenizer from Hugging Face Model Hub***

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-es"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


### ***Preprocessing function to prepare the dataset for the model***

In [ ]:
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["es"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/74776 [00:00<?, ? examples/s]

Map:   0%|          | 0/18694 [00:00<?, ? examples/s]

### ***Load Pretrained Model***

In [ ]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

### ***Apply PEFT + LoRA Configuration***

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Adjust for model type
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [8]:
peft_model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 294,912 || all params: 78,238,208 || trainable%: 0.3769


### ***Training Arguments***

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # Optional: if CUDA available
)

### ***Data Collator***

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=peft_model)

### ***Evaluation Metric: BLEU***

In [ ]:
def compute_metrics(eval_preds):
    bleu = evaluate.load("bleu")

    preds, labels = eval_preds

    # Replace -100 in labels as tokenizer.pad_token_id or ignore index
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Important: BLEU expects list of strings for preds and list of list of strings for refs
    # So wrap references inside list
    references = [[ref] for ref in decoded_labels]

    result = bleu.compute(predictions=decoded_preds, references=references)

    return {"bleu": result["bleu"]}

### ***Trainer Setup***

In [ ]:
trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2548315063.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


### ***Start Training***

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: iconicemon01 (iconicemon01-city-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Bleu
1,1.385300,1.331693,0.116382


Epoch,Training Loss,Validation Loss,Bleu
1,1.385300,1.331693,0.116382
2,1.352100,1.277694,0.113594
3,1.388500,1.264523,0.112774


TrainOutput(global_step=14022, training_loss=1.4435868097973727, metrics={'train_runtime': 11711.5834, 'train_samples_per_second': 19.154, 'train_steps_per_second': 1.197, 'total_flos': 7655160998264832.0, 'train_loss': 1.4435868097973727, 'epoch': 3.0})

### ***Saving the fine-tuned model and tokenizer***

In [14]:
peft_model.save_pretrained('fine-tuned-model')
tokenizer.save_pretrained('fine-tuned-tokenizer')

('fine-tuned-tokenizer/tokenizer_config.json',
 'fine-tuned-tokenizer/special_tokens_map.json',
 'fine-tuned-tokenizer/vocab.json',
 'fine-tuned-tokenizer/source.spm',
 'fine-tuned-tokenizer/target.spm',
 'fine-tuned-tokenizer/added_tokens.json')

### ***Loading the fine-tuned model and tokenizer for inference***

In [15]:
# Load the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned-tokenizer')
model = AutoModelForSeq2SeqLM.from_pretrained('./fine-tuned-model')

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


### ***Example input text in English***

In [16]:
input_text = "Hello, how are you today?"

In [17]:
# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

In [18]:
# Generate the translation (English to Spanish)
translated_tokens = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

In [19]:
# Decode the translated tokens back to text (Spanish)
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

In [20]:
translated_text

'Hola, ¿cómo estás hoy?'

### ***Importing additional libraries for file handling in Google Colab***

In [21]:
from google.colab import files
import shutil

In [22]:
# Create a zip file of the saved model
shutil.make_archive('/content/fine-tuned-model', 'zip', '/content', 'fine-tuned-model')

# Download the zip file
files.download('/content/fine-tuned-model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
# Create a zip file of the saved tokenizer
shutil.make_archive('/content/fine-tuned-tokenizer', 'zip', '/content', 'fine-tuned-tokenizer')

# Download the zip file
files.download('/content/fine-tuned-tokenizer.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
torch.cuda.empty_cache()  # Clears unused GPU memory